In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
url = '../src/'
df = pd.read_parquet(url+'dataset.parquet')


In [2]:
df

,date,time,user_id,event_type,product_id,category_id,first_level,second_level,third_level,brand,price,user_session
0,2019-10-01,00:00:00,554748717,view,3900821,2053013552326770905,appliances,environment,water_heater,aqua,33.20,57-6336556b0fc
1,2019-10-01,00:00:05,512742880,view,1480613,2053013561092866779,computers,desktop,desktop,pulser,908.62,a5-86594dec0db
2,2019-10-01,00:00:08,550978835,view,31500053,2053013558031024687,empty,empty,empty,luminarc,41.16,a7-abc6048498d
3,2019-10-01,00:00:11,555158050,view,2900536,2053013554776244595,appliances,kitchen,microwave,elenberg,51.46,9e-9ce44bb50ab
4,2019-10-01,00:00:22,512742880,view,1480714,2053013561092866779,computers,desktop,desktop,pulser,921.49,a5-86594dec0db
...,...,...,...,...,...,...,...,...,...,...,...,...
14052485,2019-10-31,23:59:53,566031393,view,1004708,2053013555631882655,electronics,smartphone,smartphone,huawei,151.99,5c-cee9c52ab9a
14052486,2019-10-31,23:59:55,543124499,view,1480709,2053013561092866779,computers,desktop,desktop,pulser,592.01,25-9a0d43297f9
14052487,2019-10-31,23:59:55,551943550,view,5800263,2053013553945772349,electronics,audio,subwoofer,alphard,92.67,3c-e04aa3163bb
14052488,2019-10-31,23:59:56,557202408,view,1005008,2053013555631882655,electronics,smartphone,smartphone,xiaomi,94.46,61-ac70ff0c66e


In [4]:
cpy = df[['product_id', 'category_id', 'brand', 'first_level', 'second_level',
          'third_level', 'price']].drop_duplicates('product_id').copy()

In [5]:
cpy

,product_id,category_id,brand,first_level,second_level,third_level,price
0,3900821,2053013552326770905,aqua,appliances,environment,water_heater,33.20
1,1480613,2053013561092866779,pulser,computers,desktop,desktop,908.62
2,31500053,2053013558031024687,luminarc,empty,empty,empty,41.16
3,2900536,2053013554776244595,elenberg,appliances,kitchen,microwave,51.46
4,1480714,2053013561092866779,pulser,computers,desktop,desktop,921.49
...,...,...,...,...,...,...,...
14049736,17300964,2053013553853497655,vov,empty,empty,empty,10.81
14050380,3601639,2053013563810775923,aletant,appliances,kitchen,washer,125.85
14050672,10502237,2053013553199186187,mattel,empty,empty,empty,14.39
14051063,13400769,2053013557066334713,etc,empty,empty,empty,105.54


In [6]:
cpy.set_index('product_id',inplace=True)
cpy.sort_index(inplace=True)
cpy

,category_id,brand,first_level,second_level,third_level,price
product_id,,,,,,
1000978,2053013555631882655,samsung,electronics,smartphone,smartphone,0.00
1001588,2053013555631882655,meizu,electronics,smartphone,smartphone,128.31
1001606,2053013555631882655,apple,electronics,smartphone,smartphone,363.07
1002042,2053013555631882655,samsung,electronics,smartphone,smartphone,89.81
1002062,2053013555631882655,samsung,electronics,smartphone,smartphone,77.14
...,...,...,...,...,...,...
60500005,2162513070503494350,etc,construction,tools,welding,0.00
60500006,2162513070503494350,solaris,construction,tools,welding,29.83
60500007,2162513070503494350,denzel,construction,tools,welding,37.56


In [7]:
cpy['category_id'] = cpy['category_id'].apply(lambda x : str(x))
cpy.drop('category_id',axis=1,inplace=True)

In [8]:
def con_price(x):
    if x < 10:
        return str(int(round(x)))
    elif x < 100:
        return str(int(round(x,-1)))
    elif x < 1000:
        return str(int(round(x,-2)))
    elif x < 10000:
        return str(int(round(x,-3)))

cpy['price'] = cpy['price'].apply(lambda x : con_price(x))

## 데이터가 커 한번에 분류가 안되서 1분류로 1차적으로 나눈 뒤 진행

In [9]:
first_list = cpy['first_level'].drop_duplicates().to_list()
first_list

['electronics',
 'computers',
 'empty',
 'appliances',
 'construction',
 'auto',
 'furniture',
 'kids',
 'apparel',
 'sport',
 'stationery',
 'accessories',
 'medicine',
 'country_yard']

In [10]:
for category in tqdm(first_list):
    # print(category)
    globals()['cpy_'+category] = cpy[cpy['first_level'] == category].copy()

  0%|          | 0/14 [00:00<?, ?it/s]

electronics


 57%|█████▋    | 8/14 [00:01<00:00,  8.43it/s]

computers
empty
appliances
construction
auto
furniture
kids
apparel


100%|██████████| 14/14 [00:01<00:00,  8.75it/s]

sport
stationery
accessories
medicine
country_yard


In [11]:
for category in tqdm(first_list):
    globals()['info_'+category] = []
    for i in range(len(globals()['cpy_'+category])):
        (globals()['info_'+category]).append(' '.join((globals()['cpy_'+category]).iloc[i]))

100%|██████████| 14/14 [00:05<00:00,  2.74it/s]


In [14]:
cpy_accessories.head()

,brand,first_level,second_level,third_level,price
product_id,,,,,
18300000,acer,accessories,bag,bag,90
18300003,etc,accessories,bag,bag,0
18300004,cameron,accessories,bag,bag,10
18300005,cameron,accessories,bag,bag,20
18300006,cameron,accessories,bag,bag,20
